# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

# Preprocessing the Data

In [2]:
# read csv into dataframe
filepath = "Austin Properties 2019.1.6.csv"
csv_df = pd.read_csv(filepath)

In [3]:
# check the column names
csv_df.columns

Index(['Address', 'Building Name', 'Zip Code', 'Bed', 'Bath', 'Avg SF',
       '# Units', 'Mix %', 'Units Available Units', 'Units Available Percent',
       'Avg Asking Rent/Unit', 'Avg Asking Rent/SF', 'Avg Effective Rent/Unit',
       'Avg Effective Rent/SF', 'Concessions %', 'Year Built',
       'Year Renovated', 'Units', 'Lande (Acres)', 'Vacancy %', 'Washer/Dryer',
       'Walk in Closet', 'Hardwood/Vinyl Floor', 'Walk Score', 'Transit Score',
       '1 Mile Population', '1 Mile Median Household Income',
       'Miles from Domain', 'Miles from Downtown', 'Unnamed: 29'],
      dtype='object')

In [4]:
csv_df = pd.get_dummies(data=csv_df, columns=['Washer/Dryer'])

In [5]:
# select certain variables
original_df = csv_df[['Bed', 'Bath', 'Avg SF', 'Concessions %', 'Year Built', 
   'Walk in Closet', 'Hardwood/Vinyl Floor',
   'Washer/Dryer_Yes but not in unit', 'Washer/Dryer_Yes in unit', 'Washer/Dryer_no',
   'Walk Score', 'Transit Score', '1 Mile Population', '1 Mile Median Household Income',
   'Miles from Domain', 'Miles from Downtown', 
   'Avg Effective Rent/Unit']]

In [6]:
# process na values
austin_properties_df = original_df.dropna()
# check the result of na value processing
austin_properties_df.count()

Bed                                 5468
Bath                                5468
Avg SF                              5468
Concessions %                       5468
Year Built                          5468
Walk in Closet                      5468
Hardwood/Vinyl Floor                5468
Washer/Dryer_Yes but not in unit    5468
Washer/Dryer_Yes in unit            5468
Washer/Dryer_no                     5468
Walk Score                          5468
Transit Score                       5468
1 Mile Population                   5468
1 Mile Median Household Income      5468
Miles from Domain                   5468
Miles from Downtown                 5468
Avg Effective Rent/Unit             5468
dtype: int64

In [7]:
# check the data type
austin_properties_df.dtypes

Bed                                 float64
Bath                                float64
Avg SF                              float64
Concessions %                       float64
Year Built                          float64
Walk in Closet                      float64
Hardwood/Vinyl Floor                float64
Washer/Dryer_Yes but not in unit      uint8
Washer/Dryer_Yes in unit              uint8
Washer/Dryer_no                       uint8
Walk Score                          float64
Transit Score                       float64
1 Mile Population                   float64
1 Mile Median Household Income      float64
Miles from Domain                   float64
Miles from Downtown                 float64
Avg Effective Rent/Unit             float64
dtype: object

In [8]:
# overview of the new dataframe
austin_properties_df.head()

,Bed,Bath,Avg SF,Concessions %,Year Built,Walk in Closet,Hardwood/Vinyl Floor,Washer/Dryer_Yes but not in unit,Washer/Dryer_Yes in unit,Washer/Dryer_no,Walk Score,Transit Score,1 Mile Population,1 Mile Median Household Income,Miles from Domain,Miles from Downtown,Avg Effective Rent/Unit
0,1.0,1.0,560.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1528.0
1,1.0,1.0,612.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1633.0
2,1.0,1.0,629.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1740.0
3,1.0,1.0,774.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1778.0
4,1.0,1.0,778.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1852.0


In [9]:
austin_properties_df["Year Built"].mean()

1997.693672275055

# Determine X and y for Machine Learning

In [10]:
X = austin_properties_df.iloc[:, :-1]
y = austin_properties_df['Avg Effective Rent/Unit']

# Splitting the data into training and testing sets

In [11]:
from sklearn.model_selection import train_test_split

# 80/20 training and testing data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [12]:
# check the shape of training and test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4374, 16)
(1094, 16)
(4374,)
(1094,)


# Model (I) - Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
# train the model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [15]:
# model evaluation
y_test_predicted = lin_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_predicted)
r2_test = r2_score(y_test, y_test_predicted)
print(f"Mean Squared Error (MSE): {mse_test}")
print(f"R-quared (R2): {r2_test}")

Mean Squared Error (MSE): 296512.9266368084
R-quared (R2): 0.5344241825987113


# Model (II) - Polynomial Regression

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [17]:
poly_model = Pipeline([
    ('a', PolynomialFeatures(degree=2)),
    ('b', LinearRegression(fit_intercept=False))
])

In [18]:
poly_model.fit(X_train, y_train)
poly_model.score(X_test, y_test)

0.44883007398009794

# Model (III) Random Forest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor()
param_grid = {'n_estimators' : list(range(1,999,2))}
model_rf = GridSearchCV(forest, param_grid, verbose=3)

model_rf.fit(X_train, y_train)
print(model_rf.best_params_)
print(model_rf.best_score_)

y_pred_rf = model_rf.predict(X_test)

/Users/apple/Documents/anaconda3/envs/PyData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


Fitting 3 folds for each of 499 candidates, totalling 1497 fits
[CV] n_estimators=1 ..................................................
[CV] .......... n_estimators=1, score=0.936103360114957, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8620350776106729, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8752445485214535, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.9343480771121119, total=   0.1s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.8845040595415848, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.8776823223594649, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ........

[CV] ........ n_estimators=39, score=0.9428164438377401, total=   0.7s
[CV] n_estimators=39 .................................................
[CV] ........ n_estimators=39, score=0.9314464511814243, total=   0.7s
[CV] n_estimators=39 .................................................
[CV] ........ n_estimators=39, score=0.9287848053424925, total=   0.6s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9430857404253309, total=   0.7s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9413825876316618, total=   0.7s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9227305347743747, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] ........ n_estimators=43, score=0.9449640438667319, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] .

[CV] ......... n_estimators=77, score=0.930502695942979, total=   1.2s
[CV] n_estimators=77 .................................................
[CV] ........ n_estimators=77, score=0.9355013357002917, total=   1.1s
[CV] n_estimators=79 .................................................
[CV] ........ n_estimators=79, score=0.9456394547686473, total=   1.1s
[CV] n_estimators=79 .................................................
[CV] ........ n_estimators=79, score=0.9322579683968085, total=   1.2s
[CV] n_estimators=79 .................................................
[CV] ........ n_estimators=79, score=0.9308673803336116, total=   1.3s
[CV] n_estimators=81 .................................................
[CV] ........ n_estimators=81, score=0.9406295110153147, total=   1.1s
[CV] n_estimators=81 .................................................
[CV] ........ n_estimators=81, score=0.9221225881965954, total=   1.1s
[CV] n_estimators=81 .................................................
[CV] .

[CV] ....... n_estimators=115, score=0.9323974845738037, total=   1.7s
[CV] n_estimators=117 ................................................
[CV] ....... n_estimators=117, score=0.9397100182573338, total=   1.7s
[CV] n_estimators=117 ................................................
[CV] ....... n_estimators=117, score=0.9315609773529102, total=   1.8s
[CV] n_estimators=117 ................................................
[CV] ....... n_estimators=117, score=0.9274499892984234, total=   1.7s
[CV] n_estimators=119 ................................................
[CV] ....... n_estimators=119, score=0.9464555603643081, total=   1.7s
[CV] n_estimators=119 ................................................
[CV] ....... n_estimators=119, score=0.9352693589315412, total=   1.7s
[CV] n_estimators=119 ................................................
[CV] ....... n_estimators=119, score=0.9291256370832028, total=   1.7s
[CV] n_estimators=121 ................................................
[CV] .

...

[CV] ....... n_estimators=889, score=0.9348265236010294, total=  12.9s
[CV] n_estimators=889 ................................................
[CV] ....... n_estimators=889, score=0.9302413975659783, total=  13.2s
[CV] n_estimators=891 ................................................
[CV] ....... n_estimators=891, score=0.9433153489603292, total=  12.4s
[CV] n_estimators=891 ................................................
[CV] ....... n_estimators=891, score=0.9354552123052465, total=  13.3s
[CV] n_estimators=891 ................................................
[CV] ....... n_estimators=891, score=0.9303359839102077, total=  13.6s
[CV] n_estimators=893 ................................................
[CV] ....... n_estimators=893, score=0.9423802362017597, total=  12.8s
[CV] n_estimators=893 ................................................
[CV] ....... n_estimators=893, score=0.9340042291471131, total=  12.8s
[CV] n_estimators=893 ................................................
[CV] .

[CV] ....... n_estimators=927, score=0.9310851403721302, total=  14.0s
[CV] n_estimators=929 ................................................
[CV] ....... n_estimators=929, score=0.9428782445588119, total=  13.2s
[CV] n_estimators=929 ................................................
[CV] ....... n_estimators=929, score=0.9361038823002242, total=  12.7s
[CV] n_estimators=929 ................................................
[CV] ....... n_estimators=929, score=0.9306554230248497, total=  14.2s
[CV] n_estimators=931 ................................................
[CV] ....... n_estimators=931, score=0.9424800300379463, total=  14.0s
[CV] n_estimators=931 ................................................
[CV] ....... n_estimators=931, score=0.9333670717319161, total=  14.1s
[CV] n_estimators=931 ................................................
[CV] ....... n_estimators=931, score=0.9306461960027953, total=  14.0s
[CV] n_estimators=933 ................................................
[CV] .

[CV] ....... n_estimators=967, score=0.9426098829534937, total=  13.4s
[CV] n_estimators=967 ................................................
[CV] ....... n_estimators=967, score=0.9356528611315482, total=  14.3s
[CV] n_estimators=967 ................................................
[CV] ....... n_estimators=967, score=0.9307960632032175, total=  15.6s
[CV] n_estimators=969 ................................................
[CV] ....... n_estimators=969, score=0.9430323657254022, total=  15.2s
[CV] n_estimators=969 ................................................
[CV] ....... n_estimators=969, score=0.9345112303549795, total=  14.3s
[CV] n_estimators=969 ................................................
[CV] ....... n_estimators=969, score=0.9307553210623376, total=  14.4s
[CV] n_estimators=971 ................................................
[CV] ....... n_estimators=971, score=0.9450099188476516, total=  14.6s
[CV] n_estimators=971 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done 1497 out of 1497 | elapsed: 185.7min finished


{'n_estimators': 189}
0.9387849851278849


In [21]:
import pickle
rf_model_filepath = '/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/Application/static/resources/RF999_Model.sav'
pickle.dump(model_rf, open(rf_model_filepath, 'wb'))

In [44]:
model_rf.score

<bound method BaseSearchCV.score of GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 1...959, 961, 963, 965, 967, 969, 971, 973, 975, 977, 979, 981, 983, 985, 987, 989, 991, 993, 995, 997]},
       pre_dispatch='2*n_jobs', refit=

In [ ]:
rf_model_filepath2 = '/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/Application/static/resources/Random_Forest_Model.sav'
loaded_model2 = pickle.load(open(rf_model_filepath2, 'rb'))

In [ ]:
loaded_model2.predict(X_new)

In [ ]:
loaded_model.predict(X_new)

In [23]:
X_new = [1, 1, 750, 0.02, 2018, 1, 1, 1, 0, 0, 100, 100, 70000, 50000, 0, 0]
X_new = pd.to_numeric(X_new)
X_new = np.reshape(X_new, (1, 16))
print(model_rf.predict(X_new))

[2750.38095238]


# Model (IV) - Neural Network Model

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [46]:
# define neural network model
model_nn = Sequential()
model_nn.add(Dense(12, input_dim=16, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(8, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(4, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(1, kernel_initializer='random_uniform', activation='relu'))

# compile model
model_nn.compile(loss='mse', optimizer='adam')

In [47]:
model_nn.fit(X_train, y_train, batch_size=300, epochs=10000, 
             verbose=1, 
             callbacks=[EarlyStopping(monitor='loss', min_delta=1000, patience=300)])

Epoch 1/10000
4374/4374 [==============================] - 0s 77us/step - loss: 3771290.3306
Epoch 2/10000
4374/4374 [==============================] - 0s 5us/step - loss: 3720767.3121
Epoch 3/10000
4374/4374 [==============================] - 0s 5us/step - loss: 3522664.3807
Epoch 4/10000
4374/4374 [==============================] - 0s 5us/step - loss: 2957191.4350
Epoch 5/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1929951.6612
Epoch 6/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1292998.6743
Epoch 7/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1274367.9114
Epoch 8/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1250610.6198
Epoch 9/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1242887.6701
Epoch 10/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1237321.9399
Epoch 11/10000
4374/4374 [==============================] - 0s 5us/step - loss

4374/4374 [==============================] - 0s 4us/step - loss: 465051.6271
Epoch 90/10000
4374/4374 [==============================] - 0s 4us/step - loss: 462837.6915
Epoch 91/10000
4374/4374 [==============================] - 0s 4us/step - loss: 454477.5778
Epoch 92/10000
4374/4374 [==============================] - 0s 5us/step - loss: 453985.3476
Epoch 93/10000
4374/4374 [==============================] - 0s 5us/step - loss: 431439.8667
Epoch 94/10000
4374/4374 [==============================] - 0s 5us/step - loss: 425332.6706
Epoch 95/10000
4374/4374 [==============================] - 0s 4us/step - loss: 416548.7541
Epoch 96/10000
4374/4374 [==============================] - 0s 5us/step - loss: 408587.4991
Epoch 97/10000
4374/4374 [==============================] - 0s 5us/step - loss: 401494.5488
Epoch 98/10000
4374/4374 [==============================] - 0s 5us/step - loss: 394145.7512
Epoch 99/10000
4374/4374 [==============================] - 0s 5us/step - loss: 396434.1705
Epo

4374/4374 [==============================] - 0s 4us/step - loss: 304892.8253
Epoch 178/10000
4374/4374 [==============================] - 0s 4us/step - loss: 296241.0340
Epoch 179/10000
4374/4374 [==============================] - 0s 5us/step - loss: 298195.3562
Epoch 180/10000
4374/4374 [==============================] - 0s 5us/step - loss: 296788.7665
Epoch 181/10000
4374/4374 [==============================] - 0s 4us/step - loss: 296216.1574
Epoch 182/10000
4374/4374 [==============================] - 0s 4us/step - loss: 300216.4444
Epoch 183/10000
4374/4374 [==============================] - 0s 4us/step - loss: 302499.8566
Epoch 184/10000
4374/4374 [==============================] - 0s 4us/step - loss: 294798.6992
Epoch 185/10000
4374/4374 [==============================] - 0s 4us/step - loss: 292119.5786
Epoch 186/10000
4374/4374 [==============================] - 0s 4us/step - loss: 291363.8253
Epoch 187/10000
4374/4374 [==============================] - 0s 4us/step - loss: 29408

4374/4374 [==============================] - 0s 4us/step - loss: 262635.1451
Epoch 266/10000
4374/4374 [==============================] - 0s 4us/step - loss: 265621.8835
Epoch 267/10000
4374/4374 [==============================] - 0s 4us/step - loss: 282793.6248
Epoch 268/10000
4374/4374 [==============================] - 0s 4us/step - loss: 261515.1286
Epoch 269/10000
4374/4374 [==============================] - 0s 4us/step - loss: 260504.0420
Epoch 270/10000
4374/4374 [==============================] - 0s 4us/step - loss: 270160.7166
Epoch 271/10000
4374/4374 [==============================] - 0s 4us/step - loss: 263511.0743
Epoch 272/10000
4374/4374 [==============================] - 0s 4us/step - loss: 277929.9832
Epoch 273/10000
4374/4374 [==============================] - 0s 4us/step - loss: 266256.2547
Epoch 274/10000
4374/4374 [==============================] - 0s 4us/step - loss: 262313.5648
Epoch 275/10000
4374/4374 [==============================] - 0s 4us/step - loss: 26588

4374/4374 [==============================] - 0s 4us/step - loss: 237830.7085
Epoch 354/10000
4374/4374 [==============================] - 0s 4us/step - loss: 239064.7764
Epoch 355/10000
4374/4374 [==============================] - 0s 4us/step - loss: 243961.8744
Epoch 356/10000
4374/4374 [==============================] - 0s 4us/step - loss: 244239.6522
Epoch 357/10000
4374/4374 [==============================] - 0s 5us/step - loss: 240058.9708
Epoch 358/10000
4374/4374 [==============================] - 0s 4us/step - loss: 239252.6694
Epoch 359/10000
4374/4374 [==============================] - 0s 5us/step - loss: 243168.6385
Epoch 360/10000
4374/4374 [==============================] - 0s 4us/step - loss: 241213.7909
Epoch 361/10000
4374/4374 [==============================] - 0s 4us/step - loss: 247973.9164
Epoch 362/10000
4374/4374 [==============================] - 0s 4us/step - loss: 254978.5232
Epoch 363/10000
4374/4374 [==============================] - 0s 4us/step - loss: 25138

...

4374/4374 [==============================] - 0s 5us/step - loss: 134099.9687
Epoch 5668/10000
4374/4374 [==============================] - 0s 5us/step - loss: 126079.3141
Epoch 5669/10000
4374/4374 [==============================] - 0s 4us/step - loss: 127330.9234
Epoch 5670/10000
4374/4374 [==============================] - 0s 5us/step - loss: 129149.2556
Epoch 5671/10000
4374/4374 [==============================] - 0s 5us/step - loss: 129076.1931
Epoch 5672/10000
4374/4374 [==============================] - 0s 4us/step - loss: 128635.7757
Epoch 5673/10000
4374/4374 [==============================] - 0s 5us/step - loss: 128862.4269
Epoch 5674/10000
4374/4374 [==============================] - 0s 5us/step - loss: 134198.6571
Epoch 5675/10000
4374/4374 [==============================] - 0s 5us/step - loss: 133200.8311
Epoch 5676/10000
4374/4374 [==============================] - 0s 4us/step - loss: 129402.9936
Epoch 5677/10000
4374/4374 [==============================] - 0s 4us/step - l

4374/4374 [==============================] - 0s 4us/step - loss: 140925.2589
Epoch 5755/10000
4374/4374 [==============================] - 0s 5us/step - loss: 137643.7714
Epoch 5756/10000
4374/4374 [==============================] - 0s 4us/step - loss: 128858.6951
Epoch 5757/10000
4374/4374 [==============================] - 0s 5us/step - loss: 145799.8914
Epoch 5758/10000
4374/4374 [==============================] - 0s 5us/step - loss: 162383.1574
Epoch 5759/10000
4374/4374 [==============================] - 0s 5us/step - loss: 149407.7816
Epoch 5760/10000
4374/4374 [==============================] - 0s 5us/step - loss: 130715.7516
Epoch 5761/10000
4374/4374 [==============================] - 0s 5us/step - loss: 127610.2121
Epoch 5762/10000
4374/4374 [==============================] - 0s 5us/step - loss: 135816.8379
Epoch 5763/10000
4374/4374 [==============================] - 0s 5us/step - loss: 147193.9228
Epoch 5764/10000
4374/4374 [==============================] - 0s 5us/step - l

4374/4374 [==============================] - 0s 4us/step - loss: 129414.3090
Epoch 5842/10000
4374/4374 [==============================] - 0s 4us/step - loss: 139415.9445
Epoch 5843/10000
4374/4374 [==============================] - 0s 5us/step - loss: 132328.1348
Epoch 5844/10000
4374/4374 [==============================] - 0s 5us/step - loss: 124049.0578
Epoch 5845/10000
4374/4374 [==============================] - 0s 4us/step - loss: 162547.4250
Epoch 5846/10000
4374/4374 [==============================] - 0s 5us/step - loss: 143268.7488
Epoch 5847/10000
4374/4374 [==============================] - 0s 4us/step - loss: 162972.3785
Epoch 5848/10000
4374/4374 [==============================] - 0s 5us/step - loss: 137505.2584
Epoch 5849/10000
4374/4374 [==============================] - 0s 5us/step - loss: 142654.7388
Epoch 5850/10000
4374/4374 [==============================] - 0s 4us/step - loss: 126755.4421
Epoch 5851/10000
4374/4374 [==============================] - 0s 4us/step - l

4374/4374 [==============================] - 0s 5us/step - loss: 125384.2094
Epoch 5929/10000
4374/4374 [==============================] - 0s 5us/step - loss: 126112.5409
Epoch 5930/10000
4374/4374 [==============================] - 0s 5us/step - loss: 133221.4133
Epoch 5931/10000
4374/4374 [==============================] - 0s 4us/step - loss: 137357.4182
Epoch 5932/10000
4374/4374 [==============================] - 0s 4us/step - loss: 136870.1114
Epoch 5933/10000
4374/4374 [==============================] - 0s 5us/step - loss: 138943.1459
Epoch 5934/10000
4374/4374 [==============================] - 0s 5us/step - loss: 128336.2441
Epoch 5935/10000
4374/4374 [==============================] - 0s 5us/step - loss: 126640.5942
Epoch 5936/10000
4374/4374 [==============================] - 0s 5us/step - loss: 133656.1221
Epoch 5937/10000
4374/4374 [==============================] - 0s 5us/step - loss: 133657.4268
Epoch 5938/10000
4374/4374 [==============================] - 0s 4us/step - l

4374/4374 [==============================] - 0s 5us/step - loss: 129281.3351
Epoch 6016/10000
4374/4374 [==============================] - 0s 5us/step - loss: 136545.4546
Epoch 6017/10000
4374/4374 [==============================] - 0s 4us/step - loss: 148800.8102
Epoch 6018/10000
4374/4374 [==============================] - 0s 5us/step - loss: 135981.3917
Epoch 6019/10000
4374/4374 [==============================] - 0s 4us/step - loss: 138328.0278
Epoch 6020/10000
4374/4374 [==============================] - 0s 5us/step - loss: 126266.8143
Epoch 6021/10000
4374/4374 [==============================] - 0s 6us/step - loss: 133917.0099
Epoch 6022/10000
4374/4374 [==============================] - 0s 5us/step - loss: 126517.2024
Epoch 6023/10000
4374/4374 [==============================] - 0s 5us/step - loss: 129052.4054
Epoch 6024/10000
4374/4374 [==============================] - 0s 5us/step - loss: 132886.5111
Epoch 6025/10000
4374/4374 [==============================] - 0s 5us/step - l

In [50]:
nn_model_filepath = "/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/static/resources/NN_Model.h5"
model_nn.save(nn_model_filepath)

In [52]:
loaded_model(nn_model_filepath)

TypeError: 'GridSearchCV' object is not callable

# Model (V) - K Nearest Neighbour

In [54]:
from sklearn.neighbors import KNeighborsRegressor
k_neighbour = KNeighborsRegressor()

from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : list(range(1,999,2))}
model_k_neighbour = GridSearchCV(k_neighbour, param_grid, verbose=3)

model_k_neighbour.fit(X_train, y_train)

model_k_neighbour.best_params_

model_k_neighbour.best_score_

/Users/apple/Documents/anaconda3/envs/PyData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


Fitting 3 folds for each of 499 candidates, totalling 1497 fits
[CV] n_neighbors=1 ...................................................
[CV] ........... n_neighbors=1, score=0.911738041980998, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......... n_neighbors=1, score=0.8601791831498506, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......... n_neighbors=1, score=0.9177267854641082, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .......... n_neighbors=3, score=0.9180071776410968, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] ........... n_neighbors=3, score=0.876044709779677, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .......... n_neighbors=3, score=0.9262704478831104, total=   0.0s
[CV] n_neighbors=5 ...................................................
[CV] ........

[CV] ......... n_neighbors=39, score=0.5613479221541849, total=   0.0s
[CV] n_neighbors=39 ..................................................
[CV] ......... n_neighbors=39, score=0.5270143812408595, total=   0.0s
[CV] n_neighbors=41 ..................................................
[CV] ......... n_neighbors=41, score=0.4216079041465304, total=   0.0s
[CV] n_neighbors=41 ..................................................
[CV] ......... n_neighbors=41, score=0.5505857615449867, total=   0.0s
[CV] n_neighbors=41 ..................................................
[CV] ......... n_neighbors=41, score=0.5144654394938694, total=   0.0s
[CV] n_neighbors=43 ..................................................
[CV] ......... n_neighbors=43, score=0.4174737211575657, total=   0.0s
[CV] n_neighbors=43 ..................................................
[CV] .......... n_neighbors=43, score=0.541736157928688, total=   0.0s
[CV] n_neighbors=43 ..................................................
[CV] .

[CV] ......... n_neighbors=77, score=0.4298146386345714, total=   0.0s
[CV] n_neighbors=79 ..................................................
[CV] ........ n_neighbors=79, score=0.33485629781548476, total=   0.0s
[CV] n_neighbors=79 ..................................................
[CV] ......... n_neighbors=79, score=0.4421061176136611, total=   0.0s
[CV] n_neighbors=79 ..................................................
[CV] ......... n_neighbors=79, score=0.4255377373196777, total=   0.0s
[CV] n_neighbors=81 ..................................................
[CV] ........ n_neighbors=81, score=0.33431296915420006, total=   0.0s
[CV] n_neighbors=81 ..................................................
[CV] ......... n_neighbors=81, score=0.4386399780297008, total=   0.0s
[CV] n_neighbors=81 ..................................................
[CV] ......... n_neighbors=81, score=0.4208537616684114, total=   0.0s
[CV] n_neighbors=83 ..................................................
[CV] .

[CV] ....... n_neighbors=117, score=0.31187340233237126, total=   0.0s
[CV] n_neighbors=117 .................................................
[CV] ....... n_neighbors=117, score=0.42730915409152537, total=   0.0s
[CV] n_neighbors=117 .................................................
[CV] ........ n_neighbors=117, score=0.3688868109108481, total=   0.0s
[CV] n_neighbors=119 .................................................
[CV] ....... n_neighbors=119, score=0.31077923607723157, total=   0.0s
[CV] n_neighbors=119 .................................................
[CV] ........ n_neighbors=119, score=0.4240340113828722, total=   0.0s
[CV] n_neighbors=119 .................................................
[CV] ....... n_neighbors=119, score=0.36765387531028865, total=   0.0s
[CV] n_neighbors=121 .................................................
[CV] ........ n_neighbors=121, score=0.3098896586350194, total=   0.0s
[CV] n_neighbors=121 .................................................
[CV] .

...

[CV] ....... n_neighbors=891, score=0.17992306792032875, total=   0.2s
[CV] n_neighbors=891 .................................................
[CV] ....... n_neighbors=891, score=0.13505909339884536, total=   0.2s
[CV] n_neighbors=893 .................................................
[CV] ....... n_neighbors=893, score=0.10547298417432403, total=   0.2s
[CV] n_neighbors=893 .................................................
[CV] ....... n_neighbors=893, score=0.17956053199942823, total=   0.2s
[CV] n_neighbors=893 .................................................
[CV] ........ n_neighbors=893, score=0.1348852374922085, total=   0.2s
[CV] n_neighbors=895 .................................................
[CV] ....... n_neighbors=895, score=0.10521409471102139, total=   0.2s
[CV] n_neighbors=895 .................................................
[CV] ....... n_neighbors=895, score=0.17920850250772957, total=   0.2s
[CV] n_neighbors=895 .................................................
[CV] .

[CV] ........ n_neighbors=929, score=0.1318117034063533, total=   0.2s
[CV] n_neighbors=931 .................................................
[CV] ....... n_neighbors=931, score=0.10124629935391671, total=   0.2s
[CV] n_neighbors=931 .................................................
[CV] ....... n_neighbors=931, score=0.17412506067399225, total=   0.2s
[CV] n_neighbors=931 .................................................
[CV] ........ n_neighbors=931, score=0.1316668679153281, total=   0.2s
[CV] n_neighbors=933 .................................................
[CV] ........ n_neighbors=933, score=0.1010621618034695, total=   0.2s
[CV] n_neighbors=933 .................................................
[CV] ....... n_neighbors=933, score=0.17388428771566622, total=   0.2s
[CV] n_neighbors=933 .................................................
[CV] ....... n_neighbors=933, score=0.13150545083532295, total=   0.2s
[CV] n_neighbors=935 .................................................
[CV] .

[CV] ....... n_neighbors=969, score=0.09779961823649364, total=   0.2s
[CV] n_neighbors=969 .................................................
[CV] ....... n_neighbors=969, score=0.16888061851169245, total=   0.2s
[CV] n_neighbors=969 .................................................
[CV] ........ n_neighbors=969, score=0.1286286768272712, total=   0.2s
[CV] n_neighbors=971 .................................................
[CV] ....... n_neighbors=971, score=0.09763594380824571, total=   0.2s
[CV] n_neighbors=971 .................................................
[CV] ....... n_neighbors=971, score=0.16862079653566886, total=   0.2s
[CV] n_neighbors=971 .................................................
[CV] ....... n_neighbors=971, score=0.12851135805266012, total=   0.2s
[CV] n_neighbors=973 .................................................
[CV] ....... n_neighbors=973, score=0.09747123245539879, total=   0.2s
[CV] n_neighbors=973 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done 1497 out of 1497 | elapsed:  8.6min finished


0.9067741117679614

# Residuals

In [ ]:
plt.scatter(y_test_predicted, y_test_predicted-y_test)
plt.hlines(y=0, xmin=y_test_predicted.min(), xmax=y_test_predicted.max())
plt.show()